In [ ]:
import scipy as sp
from scipy import sparse
import networkx as nx
from scipy.io import mmread
from numpy import linalg as LA
import numpy as np
import matplotlib.pyplot as plt
import heapq
import copy

In [ ]:
G = nx.read_edgelist("data.txt", nodetype=int, data=(("weight", int),))
G = nx.Graph(G)

In [ ]:
list(G.edges(data=True))

In [ ]:
def print_graph(inp_graph, ct):
    pos = nx.spring_layout(inp_graph)
    nx.draw_networkx(inp_graph, pos)
    labels = nx.get_edge_attributes(inp_graph, 'weight')
    x = nx.draw_networkx_edge_labels(inp_graph, pos, edge_labels=labels)
    plt.show("filename" + str(ct) + ".png")

In [ ]:
# Initialize
input_graph = copy.deepcopy(G)
print_graph(input_graph, 0)
no_of_nodes = input_graph.number_of_nodes()
heap = [] # store as (node, g-value)
# visited = []

In [ ]:
print(input_graph.number_of_nodes())
# print_graph(input_graph, 0)

In [ ]:
def kruskalAlgo(inp_graph):
    # stores the edges present in MST
    MST = []
    index = 0
    cost = 0
    ct = 0
    nodes = inp_graph.number_of_nodes()
    # temp graph
    graph = nx.Graph()
    # sort edges by increasing weight
    sorted_graph = sorted(inp_graph.edges(data=True), key=lambda item: item[2]['weight'])
    # MST contains exactly `V-1` edges
    i = 0
    for edgelist in sorted_graph:
        # consider the next edge with minimum weight from the graph
        
        (src, dest, weight) = edgelist
        weight = weight.get('weight')
        index = index + 1
        # add edge
        graph.add_edge(src, dest, weight=weight)
        # check for cycle
        lst = nx.cycle_basis(graph.to_undirected())
        if len(lst) == 0:
            # consider the wt if no cycle
            cost = cost + weight
            # i += 1
            # MST.append((src, dest, weight))
            # minimumCost = 0
            # N = nx.Graph()
            # print("Adding Src : " + str(src) + " Des : " + str(dest) + " Wt : " + str(weight))
            # for u, v, weight in MST:
            #     minimumCost += weight
            #     N.add_weighted_edges_from([(u, v, weight)])
            # print_graph(N, ct)
            # ct = ct + 1
        else:
            # discard the edge
            e = (src, dest, {"weight" : weight})
            graph.remove_edge(*e[:2])

        
    return sorted_graph, graph, cost

In [ ]:
#works
def get_g_n(inp_graph, src, dest, parent_g_n):
    return inp_graph[src][dest].get('weight') + parent_g_n

In [ ]:
#works
def get_h_n_and_mst(inp_graph):
    lst, graph, cost = kruskalAlgo(inp_graph)
    print("Cost of MST : ", cost)
    # print_graph(graph, 0)
    # print(lst)
    return cost

In [ ]:
#works
def remove_list_of_nodes_from_graph(inp_graph, lst):
    for i in lst:
        inp_graph.remove_node(int(i))
    return inp_graph

In [ ]:
#works
def goal_test(no_of_nodes, visited):
    return len(visited) == no_of_nodes

In [ ]:
def A_star(inp_graph, N):
    # ans
    ans = 0
    # visited list
    visited = []
    # deepcopy for aug graph
    temp = copy.deepcopy(inp_graph)
    # create augmented graph
    temp = remove_list_of_nodes_from_graph(temp, [0])
    # print_graph(input_graph, 0)
    # print_graph(temp, 0)
    # generate MST of augmented graph
    h_n = get_h_n_and_mst(temp)
    # push src node with FN = HN since GN = 0 
    heapq.heappush(heap, (h_n, h_n, 0, 0, copy.deepcopy(visited), 0))
    # std bfs jaisa
    while len(list(heap)) != 0:
        # pop node with min fn value
        f_n_parent, h_n_parent, g_n_parent, node, visited, parent = heapq.heappop(heap)
        ans = f_n_parent
        # make node visited
        visited.append(node)
        if goal_test(N, visited):
            print(visited)
            return ans
        # deepcopy for aug graph
        temp = copy.deepcopy(inp_graph)
        # create augmented graph
        temp = remove_list_of_nodes_from_graph(temp, visited)
        # generate MST of augmented graph
        h_n = get_h_n_and_mst(temp)
        # get edges of current node
        edges = []
        for u, v, weight in inp_graph.edges.data("weight"):
            if weight is not None:
                if u == node and v not in visited:
                    edges.append(v)
                elif v == node and u not in visited:
                    edges.append(u)
        print("Edges for node " + str(node) + " " + str(edges))
        # now we have all childs of node
        # push all those childs into heap
        for u in edges:
            # edge wt + parent's fn
            GN = get_g_n(inp_graph, node, u, g_n_parent)
            heapq.heappush(heap, (h_n + GN, h_n, GN, u, copy.deepcopy(visited), node))
        print("-----------------------------------------------------------------")
        print("Parent " + str(parent))
        print("Heap " + str(list(heap)))
        print("Node popped from heap " + str(node))
        print("Its visited list " + str(visited))

In [ ]:
def Start_a_star():
    temp = copy.deepcopy(input_graph)
    print_graph(input_graph, 0)
    # print(list(heap))
    # print(visited)
    # print(no_of_nodes)
    path = A_star(input_graph, no_of_nodes)
    print(path)

In [ ]:
Start_a_star()